# Evaluer bøker

Gitt en gruppe ord og grupper av dem igjen, hvordan kan vi finne bøker som matcher? Google har sine algoritmer. I dhlab er det en funksjon `nb.wordbag_eval` som henter ut grunndata for å gjøre en evaluering. Sammen med de nye fulltekstindeksene skulle det her gi noen muligheter til å bygge og analyser korpus basert på innholdsord.

Med `wordbag_eval` gis en gruppert liste med ord sammen med et sett URNer. I eksemplet under henter vi ut ord fra galaksene.

In [17]:
import dhlab as dh
import dhlab.api.dhlab_api as api
import dhlab.graph_networkx_louvain as gnl

In [79]:
import words_and_graphs as wg
import numpy as np

In [30]:
wg.relaterte_ord('hest', number=2)

,word,score
0,hest,0.398744
1,storfe,0.273908


In [31]:
war = {x:list(wg.relaterte_ord(x).word.values) for x in "våpen kjøretøy båter fly bomber soldater".split()}

In [33]:
print('\n\n'.join([x + ': ' + ', '.join(war[x]) for x in war]))

våpen: våpen, ammunisjon, forsyninger, materiell, krigsutstyr, utstyr, krigsmateriell, sprengstoff, eksplosiver, militærutstyr, personell, våpensystemer, sabotasjemateriell, proviant, våpendeler, våben, militært, rustning, rustninger, forsvarsmateriell

kjøretøy: trafikanter, kjøretøy, maskiner, bygningstekniske, forvalte, føreropplæring, et, anleggsmaskiner, kjoretoy, førere, motorvogn, kjøretoy, kjoretøy, angsten, kollektivtrafikk, hengere, trafikantar, utstyr, fotgjengere, kjoretey

båter: båter, flåter, kanoer, fly, campingvogner, båtutstyr, biler, yachter, motorer, motorkjøretøyer, påhengsmotorer, båtmotorer, redskaper, vannscootere, fiskeredskaper, skip, sykler, småbåter, redningsvester, motorbåter

fly: fly, stridsvogner, krigsskip, marinefartøyer, übåter, ubåter, helikoptre, artilleri, helikoptere, tanks, krigsskib, fartøyer, raketter, flyvere, flygere, bombefly, flydeler, skip, kampfly, pansrede

bomber: bomber, maskingevær, mitraljøser, granater, maskingeværer, giftgass, bomb

In [85]:
corpus = dh.Corpus(doctype="digibok", from_year=1950, to_year = 1990, ddk="9*", limit = 20000)

In [127]:
%%time
docs = api.evaluate_documents(war,urns = list(corpus.frame.urn.values))              

CPU times: user 948 ms, sys: 0 ns, total: 948 ms
Wall time: 5.4 s


In [128]:
id_dict = corpus.frame[['dhlabid','title']].set_index('dhlabid').astype(str).to_dict()

In [129]:
docs.index = docs.index.astype(int)

In [130]:
#docs['score'] = docs.prod(axis = 1)

In [131]:
weights = {x:1 for x in docs.columns}

In [132]:
weights

{'kjøretøy': 1, 'soldater': 1, 'båter': 1, 'fly': 1, 'våpen': 1, 'bomber': 1}

In [165]:
weights = {'kjøretøy': 10,
 'soldater': 0.01,
 'båter': 0.01,
 'fly': 1,
 'våpen': 0.01,
 'bomber': 1}

In [168]:
docs['Weighted Product'] = docs.fillna(0).apply(lambda row: (np.mean([row[col] * weights[col] for col in weights])), axis=1)

In [170]:
(docs
 .fillna(0)
 .sort_values(by="Weighted Product", ascending=False)
 .rename(index = id_dict['title'])
 .head(20)
)

,kjøretøy,soldater,båter,fly,våpen,bomber,Weighted Product
Chronique . [ 1 ],9656.0,250.0,0.0,0.0,0.0,0.0,16093.750000
Scriptores minores historiae danicae medii aevi . Vol . II,4633.0,391.0,0.0,0.0,0.0,0.0,7722.318333
Århundrets krønike,3962.0,10004.0,591.0,723.0,261.0,153.0,6767.426667
Records of the wardrobe and household . 1 : 1285-1286,3448.0,9.0,0.0,0.0,0.0,0.0,5746.681667
La défaite des sans-culottes : mouvement populaire et réaction bourgeoise en l ' an III,3277.0,1668.0,1.0,0.0,0.0,0.0,5464.448333
La défaite des sans-culottes : mouvement populaire et réaction bourgeoise en l ' an III,3276.0,1663.0,0.0,0.0,0.0,0.0,5462.771667
"Memoarer : Sovjet , Kina , Vietnam",3224.0,7304.0,97.0,254.0,335.0,106.0,5446.226667
Memoarer : de første år i Det hvite hus,2979.0,6782.0,106.0,248.0,308.0,85.0,5032.493333
Henrik Ibsen : en biografi,2926.0,5991.0,17.0,16.0,15.0,4.0,4890.038333
Ole Jacob Broch og hans samtid,2575.0,5315.0,11.0,19.0,37.0,13.0,4305.938333
